<a href="https://colab.research.google.com/github/SoniAryan3038/AI_Assignment_Repository/blob/main/Assignment_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.3 MB/s eta 0:00:00


In [2]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F


In [3]:
# Define a graph with 4 nodes and 4 edges
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)  # Adjacency matrix of the graph

# Node features (e.g., representing atom type)
x = torch.tensor([[1], [2], [3], [4]], dtype=torch.float)  # Feature matrix for atoms

# Graph Data object
data = Data(x=x, edge_index=edge_index)


In [4]:
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(1, 4)  # 1 input feature, 4 output features
        self.conv2 = GCNConv(4, 2)  # 4 input features, 2 output features

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Instantiate the model
model = GNN()


In [5]:
# Dummy labels for training (2 classes for simplicity)
labels = torch.tensor([0, 1, 0, 1], dtype=torch.long)

# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

# Train the model for 50 epochs
for epoch in range(50):
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')


Epoch 0, Loss: 0.6767
Epoch 10, Loss: 0.6331
Epoch 20, Loss: 0.6002
Epoch 30, Loss: 0.5707
Epoch 40, Loss: 0.5446


In [6]:
model.eval()
with torch.no_grad():
    out = model(data)
    print('Predictions:', out.argmax(dim=1))


Predictions: tensor([0, 0, 1, 1])
